In [1]:
%cd pruebas/indice_invertido/

/media/notebooks/pruebas/indice_invertido


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
%%writefile mapper.py
#!/usr/bin/env python3

import sys
import os

filename = os.environ.get("map_input_file", "desconocido")

for line in sys.stdin:
    line = line.strip().split()
    for word in line:
        print(f"{word}\t{filename}")

Overwriting mapper.py


In [16]:
%%writefile reducer.py
#!/usr/bin/env python3

import sys

current_word = None
docs = set()

for line in sys.stdin:
    word, doc = line.strip().split("\t", 1)
    if current_word == word:
        docs.add(doc)
    else:
        if current_word:
            print(f"{current_word}\t{','.join(sorted(docs))}")
        current_word = word
        docs = {doc}

if current_word:
    print(f"{current_word}\t{','.join(sorted(docs))}")

Overwriting reducer.py


In [17]:
%%writefile doc1.txt
el cielo es azul

Overwriting doc1.txt


In [18]:
%%writefile doc2.txt
el sol es amarillo

Overwriting doc2.txt


In [19]:
%%writefile doc3.txt
el cielo es grande

Overwriting doc3.txt


In [20]:
!hdfs dfs -ls /

Found 4 items
drwxr-xr-x   - root supergroup          0 2025-11-13 10:08 /contar_palabras
drwxr-xr-x   - root supergroup          0 2025-11-18 09:23 /indice_invertido
drwxrwx---   - root supergroup          0 2025-11-12 11:31 /tmp
drwxrwxrwt   - root root                0 2025-11-18 09:10 /yarn


In [21]:
!hdfs dfs -mkdir /indice_invertido
!hdfs dfs -put doc*.txt /indice_invertido

mkdir: `/indice_invertido': File exists
put: `/indice_invertido/doc1.txt': File exists
put: `/indice_invertido/doc2.txt': File exists
put: `/indice_invertido/doc3.txt': File exists


In [22]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper.py \
-file reducer.py \
-mapper mapper.py \
-reducer reducer.py \
-input /indice_invertido/ \
-output /indice_invertido/salida


2025-11-18 09:23:25,411 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py, /tmp/hadoop-unjar3216885619816396480/] [] /tmp/streamjob1660421496719267558.jar tmpDir=null
2025-11-18 09:23:25,955 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-11-18 09:23:26,066 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.3:8032
2025-11-18 09:23:26,211 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763455212095_0005
2025-11-18 09:23:26,566 INFO mapred.FileInputFormat: Total input files to process : 3
2025-11-18 09:23:26,644 INFO mapreduce.JobSubmitter: number of splits:3
2025-11-18 09:23:26,740 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1763455212095_0005
2025-11-18 09:23:26,740 INFO mapreduce.JobSubmitter: Executing wi

In [23]:
#!hdfs dfs -rm -r /indice_invertido/salida

In [24]:
!hdfs dfs -cat /indice_invertido/salida/part-00000

amarillo	hdfs://namenode:9000/indice_invertido/doc2.txt
azul	hdfs://namenode:9000/indice_invertido/doc1.txt
cielo	hdfs://namenode:9000/indice_invertido/doc1.txt,hdfs://namenode:9000/indice_invertido/doc3.txt
el	hdfs://namenode:9000/indice_invertido/doc1.txt,hdfs://namenode:9000/indice_invertido/doc2.txt,hdfs://namenode:9000/indice_invertido/doc3.txt
es	hdfs://namenode:9000/indice_invertido/doc1.txt,hdfs://namenode:9000/indice_invertido/doc2.txt,hdfs://namenode:9000/indice_invertido/doc3.txt
grande	hdfs://namenode:9000/indice_invertido/doc3.txt
sol	hdfs://namenode:9000/indice_invertido/doc2.txt


In [25]:
cat doc1.txt | python3 mapper.py | sort | python3 reducer.py

azul	desconocido
cielo	desconocido
el	desconocido
es	desconocido
